# Pipeline Definition

In [14]:
import pathlib
OUT_PATH = pathlib.Path("output")
OUT_PATH.mkdir(exist_ok=True)

In [15]:
import os
import glob
from dask.distributed import Client
import papermill as pm

##  Wrapper Funktion für Papermill

In [3]:
def run_notebook(template_notebook, run_id, parameter, out_path):
     pm.execute_notebook(
               input_path = f"{template_notebook}.ipynb",
               output_path = f"{out_path}/{run_id}/{template_notebook}_{parameter}.ipynb",
               parameters=dict(run_id=run_id, param=parameter)
            )

In [4]:
def run_notebooks_per_list(template_notebook, param_list, out_path, run_id, parallel=True, n_workers=None):
    
    (out_path / run_id).mkdir(parents=True, exist_ok=True)
    if n_workers is None:
        n_workers = min(4, len(param_list))
    client = Client(n_workers=n_workers)
    
    futures = []
    if parallel:
        for param in param_list:
            future = client.submit(run_notebook, template_notebook, run_id, param, out_path)
            futures.append(future)
        client.gather(futures)
            
    else: 
        for param in param_list:
            run_notebook(template_notebook, run_id, param, out_path)

## Abfolge der Notebooks

In [5]:
run_id = "example_run"

In [6]:
run_notebooks_per_list("01_Template", [5, 10, 20], OUT_PATH, run_id, parallel=True)

C:\Users\KoehlerMe\AppData\Local\Continuum\anaconda3\lib\site-packages\distributed\bokeh\core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [WinError 10048] Normalerweise darf jede Socketadresse (Protokoll, Netzwerkadresse oder Anschluss) nur jeweils einmal verwendet werden
  warnings.warn('\n' + msg)


In [7]:
run_notebooks_per_list("02_Template", [5, 10, 20], OUT_PATH, run_id, parallel=False)

Timeout waiting for IOPub output


## ausgeführte Notebooks als html ablegen

In [9]:
for notebook in glob.glob(f"{OUT_PATH / run_id}/*.ipynb"):
    os.system(f'jupyter nbconvert --to html {notebook}')